In [11]:
# 1_imports_and_config.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import re

# directories
data_dir = Path(r"..\data\processed")   # contains p1.xlsx ... p12.xlsx
out_dir = Path("./outputs")
out_dir.mkdir(exist_ok=True)

# file mapping p1..p12
files = {i: data_dir / f"p{i}.xlsx" for i in range(1,13)}
print("Data dir:", data_dir.resolve())
print("Output dir:", out_dir.resolve())


Data dir: D:\work-freelance-code\work\MacroEconomicsAnalysis-Modelling\data\processed
Output dir: D:\work-freelance-code\work\MacroEconomicsAnalysis-Modelling\notebooks\outputs


In [12]:
# 2_load_all_p_files.py
dfs = {}
for i in range(1,13):
    fp = files[i]
    try:
        dfs[i] = pd.read_excel(fp)
        print(f"Loaded p{i}: {fp.name} rows={len(dfs[i])} cols={len(dfs[i].columns)}")
    except Exception as e:
        dfs[i] = None
        print(f"Could not load p{i} ({fp.name}): {e}")


Loaded p1: p1.xlsx rows=13 cols=29
Loaded p2: p2.xlsx rows=13 cols=6
Loaded p3: p3.xlsx rows=13 cols=14
Loaded p4: p4.xlsx rows=13 cols=4
Loaded p5: p5.xlsx rows=13 cols=29
Loaded p6: p6.xlsx rows=13 cols=6
Loaded p7: p7.xlsx rows=13 cols=14
Loaded p8: p8.xlsx rows=13 cols=4
Loaded p9: p9.xlsx rows=13 cols=8
Loaded p10: p10.xlsx rows=13 cols=7
Loaded p11: p11.xlsx rows=13 cols=7
Loaded p12: p12.xlsx rows=13 cols=41


In [13]:
# 3_build_gdp_and_basic_metrics.py
# Helper: normalize year column to int
def normalize_year_col(df, col='End of period'):
    df = df.copy()
    if col not in df.columns:
        raise KeyError(f"Year column '{col}' not found in DataFrame")
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=[col]).copy()
    df[col] = df[col].astype(int)
    return df

# Ensure p1 and p5 exist
if dfs.get(1) is None or dfs.get(5) is None:
    raise SystemExit("Required files p1 or p5 missing. Load p1 and p5 in data_dir.")

p1 = normalize_year_col(dfs[1], 'End of period').rename(columns={'End of period':'Year'})
p5 = normalize_year_col(dfs[5], 'End of period').rename(columns={'End of period':'Year'})

# select core columns (GDP; bring PFCE/GFCE/GFCF/GCF/Exports/Imports from p1 if present)
p1_sel = p1[['Year','GDP']].rename(columns={'GDP':'Nominal_GDP'})
p5_sel = p5[['Year','GDP']].rename(columns={'GDP':'Real_GDP'})

for col in ['PFCE','GFCE','GFCF','GCF','Export of goods and services','Less imports of goods and services']:
    if col in p1.columns:
        p1_sel[col] = p1[col].values

# Merge and sort
gdp = pd.merge(p1_sel, p5_sel, on='Year', how='inner', validate='1:1').sort_values('Year').reset_index(drop=True)

# Compute deflator, growths, gap
gdp['Deflator'] = (pd.to_numeric(gdp['Nominal_GDP'], errors='coerce') / pd.to_numeric(gdp['Real_GDP'], errors='coerce')) * 100
gdp['Nominal_Growth_pct'] = pd.to_numeric(gdp['Nominal_GDP'], errors='coerce').pct_change() * 100
gdp['Real_Growth_pct'] = pd.to_numeric(gdp['Real_GDP'], errors='coerce').pct_change() * 100
gdp['Growth_Gap_pct'] = gdp['Nominal_Growth_pct'] - gdp['Real_Growth_pct']

# Expenditure shares where raw cols exist
for col in ['PFCE','GFCE','GFCF','GCF','Export of goods and services','Less imports of goods and services']:
    if col in gdp.columns:
        gdp[f"{col}_pctGDP"] = (pd.to_numeric(gdp[col], errors='coerce') / pd.to_numeric(gdp['Nominal_GDP'], errors='coerce')) * 100

# Net exports if both present
if ('Export of goods and services' in gdp.columns) and ('Less imports of goods and services' in gdp.columns):
    gdp['Net_Exports'] = pd.to_numeric(gdp['Export of goods and services'], errors='coerce') - pd.to_numeric(gdp['Less imports of goods and services'], errors='coerce')
    gdp['Net_Exports_pctGDP'] = (gdp['Net_Exports'] / pd.to_numeric(gdp['Nominal_GDP'], errors='coerce')) * 100

print("Built gdp DataFrame with columns:", list(gdp.columns))
print(gdp.tail())


Built gdp DataFrame with columns: ['Year', 'Nominal_GDP', 'PFCE', 'GFCE', 'GFCF', 'GCF', 'Export of goods and services', 'Less imports of goods and services', 'Real_GDP', 'Deflator', 'Nominal_Growth_pct', 'Real_Growth_pct', 'Growth_Gap_pct', 'PFCE_pctGDP', 'GFCE_pctGDP', 'GFCF_pctGDP', 'GCF_pctGDP', 'Export of goods and services_pctGDP', 'Less imports of goods and services_pctGDP', 'Net_Exports', 'Net_Exports_pctGDP']
    Year   Nominal_GDP          PFCE          GFCE          GFCF  \
8   2020  2.010359e+07  1.224536e+07  2.211933e+06  5.720386e+06   
9   2021  1.985410e+07  1.213048e+07  2.305547e+06  5.424997e+06   
10  2022  2.359740e+07  1.438270e+07  2.472153e+06  6.979647e+06   
11  2023  2.689047e+07  1.652786e+07  2.757628e+06  8.396038e+06   
12  2024  3.012296e+07  1.813043e+07  3.104298e+06  9.165224e+06   

             GCF  Export of goods and services  \
8   6.106406e+06                  3.752188e+06   
9   5.597593e+06                  3.709237e+06   
10  7.648091e+06   

In [14]:
# 4_merge_p12_cpi.py
if dfs.get(12) is None:
    print("p12 not loaded; skipping CPI merge.")
else:
    p12 = dfs[12].copy()
    # find a CPI-like column robustly
    possible = [c for c in p12.columns if re.search(r'(?i)\bCPI\b', c) or re.search(r'(?i)consumer prices', c)]
    cpi_col = None
    preferred = ['Consumer prices - CPI (Combined)', 'CPI (Combined)', 'Consumer prices - CPI Combined', 'CPI (Combined)', 'CPI']
    for name in preferred:
        if name in p12.columns:
            cpi_col = name
            break
    if cpi_col is None and possible:
        cpi_col = possible[0]
    if cpi_col:
        p12_sub = p12[['End of period', cpi_col]].rename(columns={'End of period':'Year', cpi_col:'CPI_Combined'})
        p12_sub['Year'] = pd.to_numeric(p12_sub['Year'], errors='coerce').astype('Int64').astype(int)
        gdp = pd.merge(gdp, p12_sub, on='Year', how='left')
        gdp['CPI_YoY_pct'] = pd.to_numeric(gdp['CPI_Combined'], errors='coerce').pct_change() * 100
        print("Merged CPI column from p12:", cpi_col)
    else:
        print("Could not identify CPI column in p12 automatically.")


Merged CPI column from p12: Consumer prices - CPI (Combined)


In [15]:
# 5_generate_chart_csvs.py

# ensure out_dir exists
out_dir.mkdir(parents=True, exist_ok=True)

# chart_gdp.csv
chart_gdp_fp = out_dir / "chart_gdp.csv"
df_gdp_chart = gdp[['Year','Nominal_GDP','Real_GDP']].copy()
df_gdp_chart.to_csv(chart_gdp_fp, index=False)
print("Wrote:", chart_gdp_fp)

# chart_growth_cpi.csv
chart_growth_cpi_fp = out_dir / "chart_growth_cpi.csv"
# ensure growth columns exist with consistent naming
gdp['Nominal_Growth_pct'] = gdp['Nominal_Growth_pct']
gdp['Real_Growth_pct'] = gdp['Real_Growth_pct']
if 'CPI_Combined' not in gdp.columns:
    gdp['CPI_Combined'] = np.nan
df_growth_cpi = gdp[['Year','Real_Growth_pct','Nominal_Growth_pct','CPI_Combined']].copy()
df_growth_cpi.to_csv(chart_growth_cpi_fp, index=False)
print("Wrote:", chart_growth_cpi_fp)

# chart_expenditure.csv
chart_expenditure_fp = out_dir / "chart_expenditure.csv"
df_exp = gdp[['Year']].copy()
for col_raw, out_col in [
    ('PFCE','PFCE_pctGDP'),
    ('GFCE','GFCE_pctGDP'),
    ('GFCF','GFCF_pctGDP'),
    ('Net_Exports','Net_Exports_pctGDP')
]:
    if out_col in gdp.columns:
        df_exp[out_col] = gdp[out_col]
    else:
        # if raw exists and pct not computed, attempt compute now
        if col_raw in gdp.columns:
            df_exp[out_col] = (pd.to_numeric(gdp[col_raw], errors='coerce') / pd.to_numeric(gdp['Nominal_GDP'], errors='coerce')) * 100
        else:
            df_exp[out_col] = np.nan
df_exp.to_csv(chart_expenditure_fp, index=False)
print("Wrote:", chart_expenditure_fp)

# chart_percapita.csv
chart_percapita_fp = out_dir / "chart_percapita.csv"
# find per-capita in p9 or p10 and merge; conservative string search
per_col = None
per_idx = None
for idx in [9,10]:
    dfcand = dfs.get(idx)
    if dfcand is None:
        continue
    for c in dfcand.columns:
        if re.search(r'(?i)\bper\s*capita\b', c) or re.search(r'(?i)per[_\s-]*capita|percapita', c):
            per_col = c
            per_idx = idx
            break
    if per_col:
        break

if per_col:
    dfpc = dfs[per_idx].copy()
    if 'End of period' in dfpc.columns:
        dfpc['End of period'] = pd.to_numeric(dfpc['End of period'], errors='coerce').astype('Int64').astype(int)
        dfpc = dfpc.rename(columns={'End of period':'Year'})
    dfpc[per_col] = pd.to_numeric(dfpc[per_col], errors='coerce')
    df_pc_merge = pd.merge(gdp[['Year']], dfpc[['Year', per_col]], on='Year', how='left').rename(columns={per_col:'Per_Capita'})
    df_pc_merge.to_csv(chart_percapita_fp, index=False)
    print("Wrote:", chart_percapita_fp, "(from p{})".format(per_idx))
else:
    # fallback: create Year + Per_Capita NaN
    df_pc_merge = gdp[['Year']].copy()
    df_pc_merge['Per_Capita'] = np.nan
    df_pc_merge.to_csv(chart_percapita_fp, index=False)
    print("Wrote placeholder:", chart_percapita_fp, "(per-capita not found in p9/p10)")

# chart_growths.csv
chart_growths_fp = out_dir / "chart_growths.csv"
df_growths = gdp[['Year','Real_Growth_pct','Nominal_Growth_pct']].copy()
df_growths.to_csv(chart_growths_fp, index=False)
print("Wrote:", chart_growths_fp)

print("All CSVs created in:", out_dir.resolve())


Wrote: outputs\chart_gdp.csv
Wrote: outputs\chart_growth_cpi.csv
Wrote: outputs\chart_expenditure.csv
Wrote: outputs\chart_percapita.csv (from p9)
Wrote: outputs\chart_growths.csv
All CSVs created in: D:\work-freelance-code\work\MacroEconomicsAnalysis-Modelling\notebooks\outputs


In [16]:
# 6_plot_helpers.py
def load_chart_data(fp: str, year_col: str = "Year", value_cols=None):
    if not os.path.exists(fp):
        raise FileNotFoundError(f"File not found: {fp}")
    if fp.lower().endswith(".csv"):
        df = pd.read_csv(fp)
    else:
        df = pd.read_excel(fp)
    if year_col not in df.columns:
        raise KeyError(f"Year column '{year_col}' not found. Columns: {list(df.columns)}")
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    df = df.dropna(subset=[year_col]).copy()
    df[year_col] = df[year_col].astype(int)
    if value_cols:
        missing = [c for c in value_cols if c not in df.columns]
        if missing:
            raise KeyError(f"Missing value columns: {missing}. Available: {list(df.columns)}")
        return df[[year_col] + value_cols].reset_index(drop=True)
    return df.reset_index(drop=True)

def savefig(fig, name):
    path = out_dir / name
    fig.tight_layout()
    fig.savefig(path, dpi=200)
    print("Saved:", path)


In [17]:
# 7_create_charts_from_csvs.py

# 7.1 Nominal vs Real GDP (line)
fp = out_dir / "chart_gdp.csv"
df_gdp_chart = load_chart_data(str(fp), year_col="Year", value_cols=["Nominal_GDP","Real_GDP"])
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(df_gdp_chart['Year'], df_gdp_chart['Nominal_GDP'], marker='o', label='Nominal_GDP')
ax.plot(df_gdp_chart['Year'], df_gdp_chart['Real_GDP'], marker='o', label='Real_GDP')
ax.set_title('Nominal vs Real GDP (levels)')
ax.set_xlabel('Year'); ax.set_ylabel('₹ crore')
ax.legend()
savefig(fig, "chart_nominal_vs_real_gdp.png")
plt.close(fig)

# 7.2 Growth vs CPI (dual axis)
fp = out_dir / "chart_growth_cpi.csv"
df_gc = load_chart_data(str(fp), year_col="Year", value_cols=["Real_Growth_pct","Nominal_Growth_pct","CPI_Combined"])
fig, ax1 = plt.subplots(figsize=(10,5))
ax1.plot(df_gc['Year'], df_gc['Real_Growth_pct'], marker='o', label='Real_Growth_pct')
ax1.plot(df_gc['Year'], df_gc['Nominal_Growth_pct'], marker='s', label='Nominal_Growth_pct', alpha=0.7)
ax1.set_xlabel('Year'); ax1.set_ylabel('Growth (%)')
ax2 = ax1.twinx()
ax2.plot(df_gc['Year'], df_gc['CPI_Combined'], marker='x', linestyle='--', color='tab:orange', label='CPI_Combined')
ax2.set_ylabel('CPI (index)')
ax1.set_title('GDP Growth (Real & Nominal) vs CPI')
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
savefig(fig, "chart_growth_vs_cpi.png")
plt.close(fig)

# 7.3 Expenditure composition (stacked area)
fp = out_dir / "chart_expenditure.csv"
df_exp = load_chart_data(str(fp), year_col="Year", value_cols=["PFCE_pctGDP","GFCE_pctGDP","GFCF_pctGDP","Net_Exports_pctGDP"])
# replace NaN with 0 for plotting clarity
arr = df_exp[["PFCE_pctGDP","GFCE_pctGDP","GFCF_pctGDP","Net_Exports_pctGDP"]].fillna(0).values.T
x = df_exp['Year'].values
fig, ax = plt.subplots(figsize=(10,6))
ax.stackplot(x, *arr, labels=["PFCE","GFCE","GFCF","Net_Exports"])
ax.set_title('Expenditure composition (% of Nominal GDP)')
ax.set_xlabel('Year'); ax.set_ylabel('% of GDP')
ax.legend(loc='upper left')
savefig(fig, "chart_expenditure_composition.png")
plt.close(fig)

# 7.4 Per-capita line
fp = out_dir / "chart_percapita.csv"
df_pc = load_chart_data(str(fp), year_col="Year", value_cols=["Per_Capita"])
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(df_pc['Year'], df_pc['Per_Capita'], marker='o')
ax.set_title('Per Capita Income (nominal)')
ax.set_xlabel('Year'); ax.set_ylabel('Per Capita (₹)')
savefig(fig, "chart_percapita.png")
plt.close(fig)

# 7.5 Growths distribution boxplot (Real vs Nominal)
fp = out_dir / "chart_growths.csv"
df_gs = load_chart_data(str(fp), year_col="Year", value_cols=["Real_Growth_pct","Nominal_Growth_pct"])
fig, ax = plt.subplots(figsize=(8,5))
data = [df_gs['Real_Growth_pct'].dropna().values, df_gs['Nominal_Growth_pct'].dropna().values]
ax.boxplot(data, labels=['Real_Growth_pct','Nominal_Growth_pct'])
ax.set_title('Distribution: Growth Rates (Real vs Nominal)')
savefig(fig, "chart_growths_boxplot.png")
plt.close(fig)

print("All charts generated in:", out_dir.resolve())


Saved: outputs\chart_nominal_vs_real_gdp.png
Saved: outputs\chart_growth_vs_cpi.png
Saved: outputs\chart_expenditure_composition.png
Saved: outputs\chart_percapita.png
Saved: outputs\chart_growths_boxplot.png
All charts generated in: D:\work-freelance-code\work\MacroEconomicsAnalysis-Modelling\notebooks\outputs


C:\Users\91807\AppData\Local\Temp\ipykernel_11452\1354716315.py:61: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=['Real_Growth_pct','Nominal_Growth_pct'])
